# INF 8215 - Intelligence artif.: méthodes et algorithmes 
## Automne 2018 - TP1 - Méthodes de recherche 
### Membres de l'équipe
    - Arjun Gupta (1965475)
    - Alexandre Boudreault (1486776)
    



## LE VÉLO À MONTRÉAL
Chaque année, Montréal accueille à peu près 10 millions de touristes. Soucieuse de la qualité de leur séjour, Tourisme Montréal a entamé un projet de développement d’une nouvelle application mobile afin d’assister les touristes lors de leurs déplacements dans la ville. Cette application a pour but d’aider l’utilisateur à planifier sa visite des importantes attractions de la ville, de la façon la plus efficace possible (ie, sur la durée la plus courte). Étant donné qu’il a été observé que le moyen de transport privilégié des touristes pour explorer Montréal est le vélo, cette application a pour but de générer des circuits cyclables de durée minimale. Plus précisément, étant donné une liste d’attractions munie de points de départ et d’arrivée, la tâche est de proposer, à chaque fois, un chemin qui passe par toutes les attractions indiquées une seule fois, qui débute au point de départ et qui s’achève au point d’arrivée et dont la durée de trajet est minimale.

<img src="images/montreal.png" alt="" width="800"/>

Le travail demandé dans ce TP est de développer l’algorithme interne de l’application. Nous explorerons trois mécanismes de résolution différents :
1. Définition et exploration naïve d’un arbre de recherche
2. Exploration plus efficace en utilisant l’algorithme A*
3. Optimisation locale en utilisant une métaheuristique de recherche à voisinage variable (Variable Neighborhood Search, VNS)

## PRÉSENTATION DU PROBLÈME
Une façon naturelle de représenter notre problème est d’utiliser un graphe $G=(V, A)$ dirigé et complet. Chaque sommet dans $V$ est une attraction donnée et chaque arc dans $A$ représente une piste cyclable entre deux attractions distinctes. Chaque paire de sommets $i$ et $j$ est reliée par une paire d’arcs $a_{ij}$ et $a_{ji}$ dont les poids respectifs $w(a_{ij})$ et $w(a_{ji})$ ne sont pas nécessairement égaux. Concrètement, ces poids représentent la durée du trajet d’un sommet à l’autre (ainsi, $w$ est telle que $w : A \to\mathbb R^+$).

La liste des attractions à visiter est indiquée comme la suite $P = (p_1, ..., p_m)$ où $p_1$ et $p_m$ sont les sommets de départ et d’arrivée, respectivement.

## 1. DÉFINITION ET EXPLORATION NAÏVE D’UN ARBRE DE RECHERCHE (5 points)
Définissons un arbre de recherche $\mathcal{T}$ où chaque nœud représente une solution partielle $S$. Soient $V(S) \subseteq V$ et $A(S) \subset A$ l’ensemble des sommets visités et l’ensemble des arêtes sélectionnées, respectivement. Ainsi, le coût d’une solution est donné par :
$$g(S) = \sum_{a \in A(S)} w(a)$$

Seule l’origine est visitée initialement. Ainsi, la racine de l’arbre de recherche contient une solution partielle vide $S_{\textrm{root}}$ telle que $V(S_{\textrm{root}})=\{p_1\}$ et $A(S_{\textrm{root}}) = \emptyset$.

<img src="images/tree1.png" alt="" width="100"/>

À la suite de cela, les nœuds subséquents dans l’arbre sont tous créés en ajoutant, à chaque solution partielle $S$, un sommet subséquent dans $P\backslash V(S)$ avec l’arc correspondant dans $A$ qui relie ce sommet à la dernière attraction visitée. Le sommet $p_m$ n’est ajouté qu’à la fin, lorsqu’il est le seul sommet non encore visité. Plus formellement, si on note le sommet à ajouter $c$ et le dernier sommet visité $c'$, alors la nouvelle solution partielle obtenue est $V(S) \gets V(S) \cup \{c\}$ et $A(S) \gets A(S) \cup \{(c’,c)\}$.

Ci-dessous est un exemple de l’arbre étendu depuis sa racine où $c'$ = $p_1$ :

<img src="images/tree2.png" alt="" width="400"/>

À la fin, les feuilles de l’arbre sont des solutions complètes :

<img src="images/tree3.png" alt="" width="600"/>

### 1.1 Code
La fonction fournie ci-dessous permet d’extraire d’un fichier un graphe qui répond aux spécifications détaillées plus haut. Cette fonction retourne une $\texttt{ndarray}$ ($\texttt{graph}$) de taille $|V|\times |V|$ où $\texttt{graph[i,j]}$ représente le temps nécessaire pour traverser la piste cyclable de $i$ vers $j$.



In [2]:
import numpy as np

def read_graph():
    return np.loadtxt("montreal", dtype='i', delimiter=',')

graph = read_graph()
print(graph)

[[ 0  3  8 12 15 17 20 13 18 20 23 22 20 21 21 23 26 22 25 28 29]
 [ 2  0  4 10 12 14 17  8 16 16 20 20 15 17 18 18 24 17 23 25 25]
 [ 8  4  0  2  5 10 10  3 10 11 15 12 11 14 11 14 19 13 15 20 21]
 [10 10  2  0  3  5  7  1  5  6  9  8  7  7 10 12 12 11 12 15 16]
 [14 12  4  4  0  2  6  3  1  3  6  6  5  4  5  7 10  5  9 14 13]
 [18 12  9  4  1  0  4  2  2  2  7  5  4  3  2  4  8  3  6  9 12]
 [20 16  8  7  5  2  0  7  1  1  4  2  1  1  1  1  4  2  3  9 10]
 [11  6  3  2  4  3  6  0  3  6  9  8  7  9  9 11 13  7 12 13 16]
 [19 16  9  5  1  3  1  2  0  3  6  3  1  4  4  5  6  5  7 10 12]
 [19 15  9  6  1  1  1  7  1  0  1  3  1  1  2  3  6  1  3  9 10]
 [23 20 16  7  4  8  5  7  5  2  0  2  3  3  2  1  3  1  3  4  5]
 [21 19 12  7  5  4  2  7  1  2  1  0  1  1  2  2  2  1  4  5  7]
 [19 14 11  6  6  2  2  6  1  2  1  2  0  1  1  2  4  3  6  6  7]
 [20 15 15  5  2  3  1  8  3  2  1  2  2  0  1  1  3  1  4  8  9]
 [22 18 10  8  4  2  1  9  3  1  3  1  1  2  0  3  6  1  3  6  6]
 [22 16 15

Notre première tâche est de définir la classe qui représente une solution partielle. Son constructeur est donné et reçoit comme argument la liste des sommets (attractions $P$) à visiter et le graphe ($G$). Celui-ci crée la solution $S_{\textrm{root}}$ avec les attributs suivants :
- $\texttt{g}$ : le coût de la solution partielle
- $\texttt{visited}$ : représente $V(S)$, discuté plus haut. Par définition, $\mathtt{vistited[-1]}$ représente le dernier sommet ajouté, $ c $.
- $\texttt{not}\_\texttt{visited}$ : représente $P\backslash V(S)$
- $\texttt{graph}$: représente le graphe G

Ensuite, il est demandé d’implanter la méthode $\texttt{add}$ qui mets à jour la solution partielle en ajoutant une nouvelle attraction à visiter parmi la liste $\texttt{not}\_\texttt{visited}$. Cette méthode reçoit comme arguments l’index du sommet à visiter parmi $\texttt{not}\_\texttt{visited}$ ainsi que le graphe courant.

Implantez $\texttt{add}$ :

In [61]:
import copy

class Solution:
    def __init__(self, places, graph):
        """
        places: a list containing the indices of attractions to visit
        p1 = places[0]
        pm = places[-1]
        """
        self.g = 0 # current cost
        # For part 2
        self.h = float('inf')
        
        self.graph = graph 
        self.visited = [places[0]] # list of already visited attractions
        self.not_visited = copy.deepcopy(places[1:]) # list of attractions not yet visited

        
    def add(self, idx):
        """
        Adds the point in position idx of not_visited list to the solution
        """
        self.g += graph[self.visited[-1]][self.not_visited[idx]]
        self.visited.append(self.not_visited[idx])
        self.not_visited.pop(idx)
        # For part 2
        self.h = fastest_path_estimation(self)

        
    def __lt__(self,other):
        if((self.h + self.g) < (other.h + other.g)):
            return True
        else:
            return False

La prochaine étape est d’implanter une stratégie de parcours de l’arbre de recherche. Une première méthode simple est naïve est de mettre en œuvre une recherche en largeur ([Breadth-first search](https://moodle.polymtl.ca/pluginfile.php/444662/mod_resource/content/1/recherche_en_largeur.mp4), BFS).

Implantez $\texttt{bfs}$ qui mets en œuvre cette recherche. Elle prend en arguments le graphe courant ainsi que la liste des attractions à visiter $P$ et elle retourne la meilleure solution trouvée.

In [58]:
from queue import Queue

def bfs(graph, places):
    """
    Returns the best solution which spans over all attractions indicated in 'places'
    """
    
    # This queue will contain all the solutions to be processed
    q = Queue()
    
    # We initialize the queue with the first incomplete solution : all nodes (except the first one) are not_visited
    initSolution = Solution(graph=graph, places=places)
    q.put(initSolution)
    
    # completeSolutions is a list that will stricly contain solutions where len(not_visited)==0
    completeSolutions = []
    
    # uniqueSolutions is a list that will contain all the unique solutions encountered.
    #uniqueSolutions = []
    
    
    # While there are still elements in the queue
    while(q.empty() == False):
        
        # Get the next element of the queue and place it into currentSol
        currentSol = q.get()

        # If the length of not_visited == 0, this is a complete solution
        if(len(currentSol.not_visited) == 1):
            
            # Add it to the list of complete solutions and skip to the next solution
            completeSolutions.append(currentSol)
            continue
        
        
        # For each not_visited node (except the destination node)
        for nv_index in range(0,len(currentSol.not_visited)-1):
            
            # Deepcopy the currentSol
            newSolution = copy.deepcopy(currentSol)
            
            # Add the target not_visited node to the newSolution
            newSolution.add(nv_index)
            """
            # Check if this solution is one of the solutions in uniqueSolutions
            for uniqueSolution in uniqueSolutions:
                
                # If a uniqueSolutions.visited list contains the same elements as our newSolution.visited list,
                # it means the solution has been previsouly encountered.
                if len(uniqueSolution.visited - newSolution.visited)) == 0 && len(newSolution.visited - uniqueSolution.visited) == 0:
                    
                    # So skip it to avoid re-computing the same solution multiple times.
                    continue
            """
            
            # If the solutions is not complete and it has not been encountered before,
            
            # add it to the list of uniqueSolutions
            #uniqueSolutions.append(newSolution)
            
            # add it to the queue
            q.put(newSolution)
            
    # Add the destination node
    for completeSolution in completeSolutions:
        completeSolution.add(0)
            
    # Initialize the bestSolution as the first in the list
    bestSolution = completeSolutions[0]    



    
    # Look through the list of completeSolutions
    for candidateSolution in completeSolutions:
        
        # If a candidate's cost (self.g) is lower than the cost of the current bestSolution,
        if(candidateSolution.g < bestSolution.g):
            
            # the candidateSolution becomes the bestSolution.
            bestSolution = candidateSolution
            
    
    # This block is just to visualize the best solution
    print("Best solution: %s" % bestSolution.visited)
    tot = 0
    for i in range(0,len(bestSolution.visited)-1):
        a = bestSolution.visited[i]
        b = bestSolution.visited[i+1]
        print("From %s to %s: %s + %s = %s" % (a,b,tot,bestSolution.graph[a][b],tot+bestSolution.graph[a][b]))
        tot += bestSolution.graph[a][b]
        
    
    # Return the solution with the lowest cost.      
    return bestSolution

### 1.2 Expérimentations

On propose trois exemples d’illustration pour tester notre recherche en largeur. Le premier exemple prend en compte 7 attractions, le second 10 et le dernier 11. Vu que cette recherche énumère toutes les solutions possibles, le troisième exemple risque de prendre un temps considérable à s’achever.

Mettez en œuvre ces expériences et notez le nombre de nœuds explorés ainsi que le temps de calcul requis.

In [59]:
import time 

#test 1  --------------  OPT. SOL. = 27
start_time = time.time()
places=[0, 5, 13, 16, 6, 9, 4]
sol = bfs(graph=graph, places=places)
print(sol.g)
print("--- %s seconds ---" % (time.time() - start_time))

Best solution: [0, 5, 13, 16, 6, 9, 4]
From 0 to 5: 0 + 17 = 17
From 5 to 13: 17 + 3 = 20
From 13 to 16: 20 + 3 = 23
From 16 to 6: 23 + 2 = 25
From 6 to 9: 25 + 1 = 26
From 9 to 4: 26 + 1 = 27
27
--- 0.034932851791381836 seconds ---


In [44]:
#test 2 -------------- OPT. SOL. = 30
start_time = time.time()
places=[0, 1, 4, 9, 20, 18, 16, 5, 13, 19]
sol = bfs(graph=graph, places=places)
print(sol.g)
print("--- %s seconds ---" % (time.time() - start_time))

Best solution: [0, 1, 4, 5, 9, 13, 16, 18, 20, 19]
From 0 to 1: 0 + 3 = 3
From 1 to 4: 3 + 12 = 15
From 4 to 5: 15 + 2 = 17
From 5 to 9: 17 + 2 = 19
From 9 to 13: 19 + 1 = 20
From 13 to 16: 20 + 3 = 23
From 16 to 18: 23 + 1 = 24
From 18 to 20: 24 + 3 = 27
From 20 to 19: 27 + 3 = 30
30
--- 9.12388300895691 seconds ---


In [45]:
#test 3 -------------- OPT. SOL. = 26
start_time = time.time()
places=[0, 2, 7, 13, 11, 16, 15, 7, 9, 8, 4]
sol = bfs(graph=graph, places=places)
print(sol.g)
print("--- %s seconds ---" % (time.time() - start_time))

Best solution: [0, 2, 7, 7, 9, 13, 15, 16, 11, 8, 4]
From 0 to 2: 0 + 8 = 8
From 2 to 7: 8 + 3 = 11
From 7 to 7: 11 + 0 = 11
From 7 to 9: 11 + 6 = 17
From 9 to 13: 17 + 1 = 18
From 13 to 15: 18 + 1 = 19
From 15 to 16: 19 + 4 = 23
From 16 to 11: 23 + 1 = 24
From 11 to 8: 24 + 1 = 25
From 8 to 4: 25 + 1 = 26
26
--- 110.8315007686615 seconds ---


## 2. RECHERCHE GUIDÉE À L’AIDE DE L’ALGORITHME A\* (7.5 points)
Pour notre deuxième méthode de recherche, au lieu d’énumérer toutes les solutions possibles, nous effectuons une recherche guidée à l’aide de l’algorithme A\*. Comme vu en classe, A\* est une recherche où les nœuds à explorer sont priorisés en fonction du coût courant d’une solution $g(S)$ ainsi que d’une estimation du coût restant vers la solution finale donné par une heuristique $h(S)$.

Dans le cas d’une minimisation, $h(S)$ est une borne inférieure du coût réel restant et on priorise l’exploration des nœuds dont $f(S) = g(S)+h(S)$ est le plus petit. Avec cette méthode, la première solution complète trouvée est assurément la solution optimale.

Pour une solution donnée $S$ avec un dernier sommet visité $c$, une possible fonction $h$ est telle que :

$h(S) =$ Le poids du chemin le plus court entre $c$ et $p_m$ dans le sous graphe $G_S$ contenant les sommets $P\backslash V(S) \cup \{c\}$

Remarque que ce chemin le plus court utilisé dans le calcul de l’estimation $h$ entre l’attraction courante et l’arrivée ne passera pas nécessairement pas tous les sommets restants.


Notre algorithme A\* se présente comme ceci :
1. Définir l’arbre de recherche $\mathcal{T}$ exactement comme auparavant. Le calcul de $h$ pour la solution initiale est inutile : c’est la seule solution qu’on a.
2. Sélectionner le meilleur nœud candidat pour expansion. La solution partielle $S_b$ de ce nœud candidat est telle que :

   $$ f(S_b) \leq f(S) \quad \forall S \in \mathcal{T} \qquad S_B, S \text{ pas encore sélectionnés}$$

   Si $S_b$ est une solution complète, l’algorithme s’arrête et $S_b$ est assurément la solution optimale, sinon on continue à l’étape 3.
3. Créer des solutions subséquentes qui connectent la dernière attraction visitée à chacune des attractions restantes. Attention, on ignore l’arrivée tant que celle-ci n’est pas la seule qui reste.
 - Mettez à jour les listes des sommets visités et non visités
 - Calculez $g$ et $h$ pour chaque solution
 - Insérer la nouvelle solution partielle dans l’arbre.
4. Répéter 2 et 3.


### 2.1 Code
Commençons d’abord par compléter la classe $\texttt{Solution}$ pour prendre en compte les changements nécessaires à A\* (on a besoin notamment d’un attribut supplémentaire pour l’estimation $h$).

On verra plus tard que A\* s’implante à l’aide d’une file de priorité (priority queue). Pour que celle-ci marche, il est nécessaire de surcharger (overload) l’opérateur de comparaison « < » relatif à nos objets $\texttt{Solution}$. En sachant ce qui fait qu'une solution est meilleure qu’une autre pour l'exploration, implanter la méthode $\_\_\texttt{lt}\_\_$ dans $\texttt{Solution}$. Son prototype est $\_\_\texttt{lt}\_\_\texttt{(self, other)}$.

Maintenant, nous devons implanter la fonction d’estimation $h$. Pour cela, on utilise l’[algorithme de Dijkstra](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm) pour trouver le chemin le plus court entre la dernière attraction visitée $c$ et l’arrivée $p_m$. Il est possible d’adapter cet algorithme pour qu’il s’arrête dès que le chemin le plus court entre c et pm est trouvé.

**Prescriptions d’implantation :**
- Appliquer Dijkstra pour trouver le chemin le plus court entre $c$ et $p_m$
- Retourner le poids de ce chemin

In [60]:
def fastest_path_estimation(sol):
    if(len(sol.not_visited) == 0):
        return 0
    """
    Returns the time spent on the fastest path between 
    the current vertex c and the ending vertex pm
    """
    c = sol.visited[-1]         # Most recently visited node (11)
    pm = sol.not_visited[-1]    # Last node to visit (4)
    
    # Initialize unvisited node list
    unvisited_dict = {}
    
    # For each unvisited node (8,4)
    for n in sol.not_visited:
        
        # append a tuple for that node, with a distance of infinity, and the previous node (being None)
        unvisited_dict[n] = (float('inf'),None)
    
    # Initialize visited node dictionary
    visited_dict = {}
    
    # Append a tuple for the starting node, with a distance of 0, and the previous node as itself
    visited_dict[c] = (0,c)
    
    # Initialize current position
    current_node_key = c

    while pm not in visited_dict:
        
        # Loop through all univisited nodes
        for unvisited_node_key in unvisited_dict:
            
            
            # If the sum of the current_node's distance and the edge between the current_node and the unvisited_node is smaller than 
            #       the current distance of the unvisited node
            if(sol.graph[current_node_key][unvisited_node_key] + visited_dict[current_node_key][0] < unvisited_dict[unvisited_node_key][0]):

                # Replace the tuple with the following
                #    (distance of current_node + edge length between current node and univisted_node ,
                #     current_node_key )
                unvisited_dict[unvisited_node_key] = (sol.graph[current_node_key][unvisited_node_key] + visited_dict[current_node_key][0],current_node_key)
                
        
        # Initialize the closest unvisited_node to None
        closest_unvisited_node_key = None
        
        # Re-loop through the unvisited_nodes to identify the closest node
        for unvisted_node_key in unvisited_dict:
            
            # If the current unvisited node is closer than the previous closest
            if(closest_unvisited_node_key == None or unvisited_dict[closest_unvisited_node_key][0] > unvisited_dict[unvisted_node_key][0]):
                
                # The current unvisited_node_key becomes the closest_unvisited_node_key
                closest_unvisited_node_key = unvisted_node_key

        

        # Update the current_node_key
        current_node_key = closest_unvisited_node_key
        
        # Add the closest node to the visited
        visited_dict[closest_unvisited_node_key] = unvisited_dict[closest_unvisited_node_key]
        
        # Remove the current_node from the unvisited list
        del unvisited_dict[closest_unvisited_node_key]
        
        
    # Following block is for visualization
    """
    print("Solution:(%s)" % visited_dict[pm][0], end='')
    cur = pm
    while(cur != c):
        print("[cur(%s) -> prev(%s)] " % (cur,visited_dict[cur][1]),end='')
        cur = visited_dict[cur][1]
        
    print()
    """
 
    # Return the distance to the pm node
    return visited_dict[pm][0]
    
    
    


Finalement, il est temps d’implanter A\*. On aura besoin d’une file de priorité qui retournera toujours le meilleur nœud candidat de $\mathcal{T}$ pour l’étendre (l’opérateur surchargé de comparaison assure cela).

**Prescriptions d’implantation (cf. détail des étapes de l’algorithme plus haut) :**
- Tant que les solutions extraites de la file de priorité ne sont pas complètes :
  *	Sélectionner et étendre le nœud extrait de la file comme détaillé plus haut
  * Calculer $g$ et $h$ pour chaque nouvelle solution partielle obtenue
  * Remettre ces solutions dans la file
- Retourner la première solution complète extraite de la file (c’est la solution optimale)

In [63]:
import heapq

def A_star(graph, places):
    """
    Performs the A* algorithm
    """

    # blank solution
    root = Solution(graph=graph, places=places)

    # search tree T
    T = []
    heapq.heapify(T)
    heapq.heappush(T, root)
    
    # While there are still solutions to be explored
    while(len(T) > 0):
        
        # Get the one where g(s) + h(s) is the lowest (highest priority in heap)
        current_sol = heapq.heappop(T)
        
        # This block is simply for visualisation
        """
        print("visited: \t\t",end='')
        print(current_sol.visited)
        print("not_visited: \t\t",end='')
        print(current_sol.not_visited)
        print("g(%s) + h(%s) = f(%s)" % (current_sol.g,current_sol.h,current_sol.g+current_sol.h))
        print("****************************************")
        """

            
        if len(current_sol.not_visited) == 0:
            break
            
        # If the not_visited list is empty (except for the final node), it means that it is a complete solution
        elif len(current_sol.not_visited) == 1:
            
            # Add the final stop to the current_solution
            current_sol.add(0)
            heapq.heappush(T,current_sol)
            continue
            
        
        # For each index of the not_visited list in the current_solution
        for i in range(0, len(current_sol.not_visited) - 1):
            
            # Create a copy of the current_solution
            new_solution = copy.deepcopy(current_sol)
            
            # Add the current node to the copy
            new_solution.add(i)
            #print("Solution: %s -> %d + %d = %d" % (new_solution.visited,new_solution.g,new_solution.h, new_solution.g+new_solution.h))
            
            # Add the copy to the heap
            heapq.heappush(T,new_solution)
            
    # This block is just to visualize the best solution
    print("Best solution: %s" % current_sol.visited)
    tot = 0
    for i in range(0,len(current_sol.visited)-1):
        a = current_sol.visited[i]
        b = current_sol.visited[i+1]
        print("From %s to %s: %s + %s = %s" % (a,b,tot,current_sol.graph[a][b],tot+current_sol.graph[a][b]))
        tot += current_sol.graph[a][b]
        
    return current_sol


### 2.2 Expérimentations

On ajoute un Quatrième exemple d’exécution avec 15 attractions. Là encore, mettez en œuvre ces expériences avec le nouvel algorithme A\* conçu et notez le nombre de nœuds explorés ainsi que le temps de calcul requis.

In [64]:
#test 1  --------------  OPT. SOL. = 27
start_time = time.time()
places=[0, 5, 13, 16, 6, 9, 4]
astar_sol = A_star(graph=graph, places=places)
print(astar_sol.g)
print(astar_sol.visited)
print("--- %s seconds ---" % (time.time() - start_time))

Best solution: [0, 5, 13, 16, 6, 9, 4]
From 0 to 5: 0 + 17 = 17
From 5 to 13: 17 + 3 = 20
From 13 to 16: 20 + 3 = 23
From 16 to 6: 23 + 2 = 25
From 6 to 9: 25 + 1 = 26
From 9 to 4: 26 + 1 = 27
27
[0, 5, 13, 16, 6, 9, 4]
--- 0.01866888999938965 seconds ---


In [65]:
#test 2  --------------  OPT. SOL. = 30
start_time = time.time()
places=[0, 1, 4, 9, 20, 18, 16, 5, 13, 19]
astar_sol = A_star(graph=graph, places=places)
print(astar_sol.g)
print(astar_sol.visited)
print("--- %s seconds ---" % (time.time() - start_time))

Best solution: [0, 1, 4, 5, 9, 13, 16, 18, 20, 19]
From 0 to 1: 0 + 3 = 3
From 1 to 4: 3 + 12 = 15
From 4 to 5: 15 + 2 = 17
From 5 to 9: 17 + 2 = 19
From 9 to 13: 19 + 1 = 20
From 13 to 16: 20 + 3 = 23
From 16 to 18: 23 + 1 = 24
From 18 to 20: 24 + 3 = 27
From 20 to 19: 27 + 3 = 30
30
[0, 1, 4, 5, 9, 13, 16, 18, 20, 19]
--- 0.16869401931762695 seconds ---


In [67]:
#test 3  --------------  OPT. SOL. = 26
start_time = time.time()
places=[0, 2, 7, 13, 11, 16, 15, 7, 9, 8, 4]
astar_sol = A_star(graph=graph, places=places)
print(astar_sol.g)
print(astar_sol.visited)
print("--- %s seconds ---" % (time.time() - start_time))

Best solution: [0, 2, 7, 7, 9, 13, 15, 16, 11, 8, 4]
From 0 to 2: 0 + 8 = 8
From 2 to 7: 8 + 3 = 11
From 7 to 7: 11 + 0 = 11
From 7 to 9: 11 + 6 = 17
From 9 to 13: 17 + 1 = 18
From 13 to 15: 18 + 1 = 19
From 15 to 16: 19 + 4 = 23
From 16 to 11: 23 + 1 = 24
From 11 to 8: 24 + 1 = 25
From 8 to 4: 25 + 1 = 26
26
[0, 2, 7, 7, 9, 13, 15, 16, 11, 8, 4]
--- 0.4915168285369873 seconds ---


In [ ]:
#test 4  --------------  OPT. SOL. = 40
start_time = time.time()
places=[0, 2, 20, 3, 18, 12, 13, 5, 11, 16, 15, 4, 9, 14, 1]
astar_sol = A_star(graph=graph, places=places)
print(astar_sol.g)
print(astar_sol.visited)
print("--- %s seconds ---" % (time.time() - start_time))

Best solution: [0, 3, 5, 13, 15, 18, 20, 16, 11, 12, 14, 9, 4, 2, 1]
From 0 to 3: 0 + 12 = 12
From 3 to 5: 12 + 5 = 17
From 5 to 13: 17 + 3 = 20
From 13 to 15: 20 + 1 = 21
From 15 to 18: 21 + 1 = 22
From 18 to 20: 22 + 3 = 25
From 20 to 16: 25 + 2 = 27
From 16 to 11: 27 + 1 = 28
From 11 to 12: 28 + 1 = 29
From 12 to 14: 29 + 1 = 30
From 14 to 9: 30 + 1 = 31
From 9 to 4: 31 + 1 = 32
From 4 to 2: 32 + 4 = 36
From 2 to 1: 36 + 4 = 40
40
[0, 3, 5, 13, 15, 18, 20, 16, 11, 12, 14, 9, 4, 2, 1]
--- 118.64444899559021 seconds ---


### 2.3 Une meilleure borne inférieure

Notre algorithme A\* est déjà beaucoup plus efficace qu’une recherche naïve. Cependant, la qualité de l’heuristique $h$ a un très grand impact sur la vitesse de A\*. Une heuristique plus serrée devrait accélérer A\* de façon significative. Notre estimation $h$ basée sur Dijkstra est très large à cause du fait qu’elle ne considère pas toutes les attractions restantes.

Une meilleure heuristique pourrait être basée sur la **Spanning Arborescence of Minimum Weight** qui s’apparente à une Minimum Spanning Tree pour graphes orientés. On propose de construire une telle Spanning Arborescence sur le reste des attractions $P\backslash V(S) \cup \{c\}$. Ici la racine est la dernière attraction visitée $c$. Une façon classique de résoudre ce problème est d’utiliser l’[algorithme de Edmonds](https://en.wikipedia.org/wiki/Edmonds%27_algorithm).

Implantez cet algorithme et refaites les expériences avec A\* en utilisant cette nouvelle heuristique :

In [ ]:
def minimum_spanning_arborescence(sol):
    """
    Returns the cost to reach the vertices in the unvisited list 
    """

## 3. RECHERCHE LOCALE À VOISINAGE VARIABLE  (7.5 points)

Cette fois-ci, au lieu de construire une solution optimale depuis une solution vide, on commence d’une solution complète, non-optimale, qu’on améliore à l’aide d’une recherche locale en utilisant une recherche locale à voisinage variable ([Variable Neighborhood Search](https://en.wikipedia.org/wiki/Variable_neighborhood_search), VNS).

<img src="images/vns.png" alt="" width="800"/>

### 3.1 Code

On commence par créer une solution initiale. Celle-ci est une suite ordonnée des attractions de $p_1$ à $p_m$ dans $P$. Pour cela, on fait appel à une [recherche en profondeur (Depth-First Search, DFS)](https://moodle.polymtl.ca/pluginfile.php/445484/mod_resource/content/1/recherche_en_profondeur.mp4) qu’on arrête aussitôt qu’une solution complète est trouvée. Pour aider à diversifier la recherche, la méthode permettant de générer une solution initiale peut être randomisée de telle sorte que l'algorithme VNS puisse lancer la recherche dans différentes régions de l'espace solution. Ainsi, dans la fonction DFS, la sélection de l'enfant pour continuer la recherche doit être aléatoire.

**Prescriptions d’implantation :**
- Mettre en œuvre une recherche en profondeur
- Créer un objet $\texttt{Solution}$ relatif à cette solution
- Ajuster les attributs de cet objet avec les bonnes valeurs de coûts et d’attractions visitées
- Retourner la solution trouvée.

In [ ]:
from random import shuffle, randint

def initial_sol(graph, places):
    """
    Return a completed initial solution
    """
    return dfs(graph,places)
    
    
def dfs(graph, places):
    """
    Performs a Depth-First Search that selects children randomly:
        - stop is the number of solutions found after which the algorithm should stop (1 if we
        only want the first solution found)
        
    Returns the first solution found by the DFS
    """
    
    solution = Solution(graph=graph, places=places)
    
    # Select at random an index from not_visited except the last one which is the end point selected by the user
    while len(solution.not_visited) > 1:
        solution.add(randint(0,len(solution.not_visited)-1))
    
    solution.add(0)
    
    return solution

Pour définir une VNS, il faut définir les $k_\textrm{max}$ voisinages de recherche locale possibles. Pour notre problème, une bonne et simple répartition des voisinages est telle qu’un voisinage $k$ correspond à la permutation de $k$-paires de sommets dans $V(S)$.

On appelle **shaking** l’étape de génération d’une solution dans le voisinage $k$. Le travail qui suit correspond à l’implantation de cette étape. $\texttt{shaking}$ admet 3 arguments que sont la solution de départ, l’indice du voisinage $k$ ainsi que le graph courant.

Attention, avant d’implanter $\texttt{shaking}$, il est nécessaire de créer une méthode $\texttt{swap}$ dans la classe $\texttt{Solution}$. Cette méthode permet de mettre en œuvre la permutation dans une solution donnée (en mettant à jour tous les attributs nécessaires pour que la solution soit cohérente).

**Prescriptions d’implantation de shaking :**
- Sélectionner au hasard deux indices $i$ et $j$ différents et tels que $i, j \in \{2,...,m-1\}$
- Faire une copie de la solution courante et faire la permutation
- Retourner la solution créée

In [ ]:
def shaking(sol, k, graph):
    """
    Returns a solution on the k-th neighrboohood of sol
    """
    
    # Create a list with indexes ranging from the 2nd index to the penultimate
    indexes = range(1,len(sol.visited)-1)
    # Randomize the list
    shuffle(indexes)
    
    # Creating a copy of the solution
    newSolution = copy.deepcopy(sol)
    
    # While there are at least two elements in the list and until we have k pairs i and j
    while len(indexes) > 1:
        # Extract the last two elements of the already random list, these represent one pair of indexes
        i = indexes.pop()
        j = indexes.pop()
        newSolution.swap(i,j,graph)

        # Decrementing the number of pairs selected. If we don't have anymore pairs to select: break
        k -= 1
        if k == 0:
            break    
    
    return newSolution


Une dernière étape essentielle dans une VNS est l’application d’un algorithme de recherche locale à la solution issue du shaking. Pour cela, on propose la recherche locale 2-opt. Celle-ci intervertit deux arcs dans la solution, à la recherche d’une qui est meilleure.

Pour un sommet $ i $, soit $ i '$ le successeur immédiat de $ i $ dans la séquence $ V (S) $. L'algorithme 2-opt fonctionne comme suit: pour chaque paire de sommets non consécutifs $ i, j $, vérifiez si en échangeant la position des sommets $ i '$ et $ j $ entraîne une amélioration du coût de la solution. Si oui, effectuez cet échange. Ce processus se répète jusqu'à ce qu'il n'y ait plus d'échanges rentables. On réalise cette opération pour toutes les paires d’arcs éligibles à la recherche du plus petit coût.

<img src="images/2opt.png" alt="" width="800"/>

<img src="images/2opt2.png" alt="" width="800"/>


Implantez $\texttt{local}\_\texttt{search}\_\texttt{2opt}$. 

**Prescriptions d’implantation :**
- Considérer chaque paire d’indices $i = \{2,..,m-3\}$ and $j = \{i+2, m-1\}$
- Si l’échange donne un plus bas coût, on le réalise
- Répéter jusqu’à optimum local.

In [ ]:
def local_search_2opt(sol):
    """
    Apply 2-opt local search over sol
    """


Finalement, il est temps d'implanter notre VNS. La méthode $\texttt{vns}$ reçoit une solution complète, le graphe courant, le nombre maximal de voisinages et un temps de calcul limite. Celle-ci retourne la solution optimale trouvée

**Prescriptions d’implantation :**
- À chaque itération, la VNS génère une solution dans le k-ème voisinage (shaking) à partir de la meilleure solution courante et applique une recherche locale 2-opt dessus
- Si la nouvelle solution trouvée a un meilleur coût, mettre à jour la meilleure solution courante
- Répéter le processus jusqu'à $\texttt{t}\_\texttt{max}$

In [ ]:
def vns(sol, k_max, t_max):
    """
    Performs the VNS algorithm
    """

### 3.2 Experiments

Mettez en oeuvre la VNS sur les exemples d'illustration suivants et raportez les solutions obtenue:

In [ ]:
# test 1  --------------  OPT. SOL. = 27
places=[0, 5, 13, 16, 6, 9, 4]
sol = initial_sol(graph=graph, places=places)
start_time = time.time()
vns_sol = vns(sol=sol, k_max=10, t_max=1)
print(vns_sol.g)
print(vns_sol.visited)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
#test 2  --------------  OPT. SOL. = 30
places=[0, 1, 4, 9, 20, 18, 16, 5, 13, 19]
sol = initial_sol(graph=graph, places=places)

start_time = time.time()
vns_sol = vns(sol=sol, k_max=10, t_max=1)
print(vns_sol.g)
print(vns_sol.visited)

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# test 3  --------------  OPT. SOL. = 26
places=[0, 2, 7, 13, 11, 16, 15, 7, 9, 8, 4]
sol = initial_sol(graph=graph, places=places)

start_time = time.time()
vns_sol = vns(sol=sol, k_max=10, t_max=1)
print(vns_sol.g)
print(vns_sol.visited)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# test 4  --------------  OPT. SOL. = 40
places=[0, 2, 20, 3, 18, 12, 13, 5, 11, 16, 15, 4, 9, 14, 1]
sol = initial_sol(graph=graph, places=places)

start_time = time.time()
vns_sol = vns(sol=sol, k_max=10, t_max=1)
print(vns_sol.g)
print(vns_sol.visited)
print("--- %s seconds ---" % (time.time() - start_time))

## 4. BONUS (1 point)

Expliquez dans quelle situation chacun des algorithmes développés est plus approprié (prenez en compte l’évolutivité du problème)